## JSON 만들기

In [ ]:
#-*-coding: utf-8
# 시장/ 부시장 직속 부서들만 먼저!
json_hierarchy = [
    {
        "div" : "서울특별시장"
    },
    {
        "div" : "대변인",
        "child" : "언론담당관"
    },
    {
        "div": "감사위원회",
        "child": "조사담당관/안전담당관/감사담당관"
    },
    {
        "div": "시민감사옴부즈만위원회"
    },
    {
        "div": "서울혁신기획관",
        "child": "인권담당관/지역공동체담당관/사회혁신담당관/청년정책담당관/갈등조정담당관/민관협력담당관"
    },
    {
        "div": "시민소통기획관",
        "child": "시민소통담당관/시민봉사담당관/도시브랜드담당관/신속행정담당관/뉴미디어담당관"
    },
    {
        "div": "행정1부시장"
    },
    {
        "div": "여성가족정책실",
        "child": "가족담당관/외국인다문화담당관/여성정책담당관/보육담당관"
    },
    {
        "div": "민생사법경찰단"
    },
    {
        "div": "비상기획관",
        "child": "민방위담당관"
    },
    {
        "div": "정보기획관",
        "child": "통계데이터담당관/정보기획담당관/정보통신보안담당관/공간정보담당관/정보시스템담당관"
    },
    {
        "div": "일자리노동정책관",
        "child": "사회적경제담당관/노동정책담당관/일자리정책담당관"
    },
    {
        "div": "행정2부시장"
    },
    {
        "div": "도시공간개선단"
    },
    {
        "div": "기술심사담당관"
    },
    {
        "div": "정무부시장"
    },
   {
       "div": "기획조정실",
       "child": [
           {
               "div": "국제교류담당관"
           },
           {
               "div": "정책기획관",
               "child": "기획담당관/법률지원담당관/대외협력담당관/조직담당관/법무담당관/평가담당관"
           },
           {
               "div": "해외도시협력담당관"
           },
           {
               "div": "재정기획관",
               "child": "예산담당관/시민참여예산과/공기업담당관/재정관리담당관/시민참여예산담당관"
           }
       ]
    },
    {
       "div": "경제진흥본부",
       "child": {
           "div": "창조경제기획관",
           "child": "도시농업과/경제정책과/민생경제과/화융합경제과/디지털창업과/투자유치과/소상공인지원과"
       }
    },
    {
          "div": "복지본부",
          "child":"자활지원과/어르신복지과/희망복지지원과/복지정책과/장애인복지정책과/인생이모작지원과/장애인자립지원과"
       },

       {
          "div": "문화본부",
          "child":[
            {
              "div": "문화정책과"
            },
            {
              "div": "한양도성도감"
            },
            {
              "div": "문화시설추진단",
              "child": "문화시설과/박물관"
            },
            {
              "div": "역사문화재과"
            },
            {
              "div": "디자인정책과"
            },
            {
              "div": "문화예술과"
            }
          ]
       },
       {
          "div": "기후환경본부",
          "child":"자원순환과/대기정책과/기후대기과/에너지시민협력과/녹색에너지과/생활환경과/대기관리과/환경정책과/기후변화대응과"
       },
       {
          "div": "행정국",
          "child":"인사과/인력개발과/정보공개정책과/총무과/자치행정과"
       },
       {
          "div": "재무국",
          "child": "38세금징수과/계약심사과/재무과/세무과/자산관리과/세제과"
       },
       {
         "div": "평생교육국",
         "child": "평생교육과/청소년정책과/교육정책과/친환경급식"
       },
       {
         "div": "관광체육국",
         "child": "관광정책과/체육진흥과/체육정책과/관광사업과"
       },
       {
         "div": "시민건강국",
         "child": "식품안전과/생활보건과/식품정책과/건강증진과/동물보호과/보건의료정책과"
       },
       {
         "div": "안전총괄본부",
         "child":[
           {
             "div":"서울역일대종합발전기획단"
           },
           {
              "div":"안전총괄관",
              "child": "안전총괄과/시설안전과/도로관리과/교량안전과/도로시설과/도로계획과/보도환경개선과/상황대응과"
           }
         ]
       },
    ]


## JSON을 이용해 url 만들기

In [ ]:
from urllib import parse

# URL 추출
urls = [];
tail = '&dept%5B%5D='

# {} dict 
# [] list

def extract_value(data, origin_url):
    if(isinstance(data, list)):
        for single_list in data:
            extract_value(single_list, origin_url)
    elif(isinstance(data, dict)):
        #dict {} 일 경우
        if(len(data) == 1):
            if(origin_url != ''):
                origin_url = origin_url + tail
            div = data.get('div')
            url = origin_url + parse.quote(div)
            urls.append(url)
        else:
            div = data.get('div')
            child = data.get('child', 'empty')
            if origin_url != '':
                origin_url = origin_url + tail
            extract_value(child, origin_url + parse.quote(div))
    elif(isinstance(data, str)):
        divs = data.split('/')
        for single_div in divs:
            url = origin_url + tail + parse.quote(single_div)
            urls.append(url)

extract_value(json_hierarchy, "")

## 크롤링

In [1]:
# 필요한 라이브러리 가져오기
from urllib import request
!pip install html5lib
import html5lib
!pip install lxml
import pandas as pd
import numpy as np

In [ ]:
import re
df = pd.DataFrame()
base_url_head = "https://opengov.seoul.go.kr/expense?dept%5B%5D=%EC%84%9C%EC%9A%B8%EC%8B%9C%EB%B3%B8%EC%B2%AD&dept%5B%5D="
url_2017 = "&year=2017&month="
url_2018 = "&year=2018&month="
column_div = "1소속부서"

# urls돌면서 데이터 가져오기
for div in urls:
  div_url = (base_url_head + div)
  # unquote_div : url 첫부분 가져와서 
  unquote_div = parse.unquote(div)
  div_name = re.search('([가-힣]*)&de', unquote_div)
  if div_name is None:
    div_name = unquote_div
  else:
    div_name = div_name.group(1)
    
  for month in range(1,9): 
    url_month = div_url + url_2018 + str(month)
    try:
      df_month = pd.read_html(url_month)[1]
      # decode된 url의 첫 부분을 1소속부서 column에 넣어주기
      df_month[column_div] = div_name
      df = df.append(df_month)
      print(url_month)
    except IndexError:
      print('err')
    
  for month in range(8,13): 
    url_month = div_url + url_2017 + str(month)
    try:
      df_month = pd.read_html(url_month)[1]
      # decode된 url의 첫 부분을 1소속부서 column에 넣어주기
      df_month[column_div] = div_name
      df = df.append(df_month)
      print(url_month)
    except IndexError:
      print('err')
    

크롤링 하는데 엄청 오래걸리네...

## 데이터 확인하기

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
df.head(2)

In [ ]:
df.tail(2)

## Tidy 1차 : column 분리하기

In [ ]:
# 집행장소와 주소를 분리하자
df_copy = df.copy()
df_copy['집행지주소'] = df_copy['집행장소'].str.extract(r'\((.*)\)')
df_copy['집행지주소'] = df_copy['집행지주소'].replace(' ', '')
df_copy['집행지명'] = df_copy['집행장소'].str.extract(r'(.*)\(')
df_copy['집행지명'] = df_copy['집행지명'].replace(' ', '')
del df_copy['집행장소']
df_copy.head(3)

In [ ]:
# 필요없는 Column 없애기
del df_copy['연번']
del df_copy['부서명']
df_copy.head(3)

In [ ]:
# 집행일시를 년/월/일/시간으로 분리하자
df_copy['year'] = df_copy['집행일시'].str.extract(r'(20..)')
df_copy['month'] = df_copy['집행일시'].str.extract(r'-(\d\d)-')
df_copy['day'] = df_copy['집행일시'].str.extract(r'-(\d\d\s)')
df_copy['time'] = df_copy['집행일시'].str.extract(r'(\d?\d:\d?\d)')
df_copy.head(3)

In [ ]:
del df_copy['집행일시']
df_copy.head(3)

In [ ]:
# '대변인' 테이블만 보기
df_copy[df_copy['1소속부서'] == '대변인'].head(3)

## 파일로 저장하기

In [ ]:
# 크롤링하는데 오래 걸리니 데이터를 파일로 저장해두자
from IPython.display import FileLink, FileLinks

df_copy.to_csv('C:\workspace\djangoPrj\popular_restaurants_from_officials/raw.csv', index=False)
# df_copy.to_excel('C:\workspace\djangoPrj\popular_restaurants_from_officials/seoul_dining_crawl.xlsx', index=False)

FileLinks('C:\workspace\djangoPrj\popular_restaurants_from_officials')

## Top20 집계

In [ ]:
df_copy = pd.read_csv('raw.csv')

In [ ]:
df_copy.shape

In [ ]:
df_copy['cnt'] = 1
df_copy.groupby('집행지명').sum().sort_values(by='cnt', ascending=False)[:5]

- 참숯골, 우도일식, 구이구이, 곰국시집이 여러개다. 왜지...?

In [ ]:
name_arr = df_copy['집행지명'].unique()
np.set_printoptions(threshold=np.nan)
name_arr

unique로 찍어보면, 아직 띄어쓰기가 제대로 제거되지 않았다.
<br>
```df_copy['집행지명'] = df_copy['집행지명'].replace(' ', '')```
를 수행해주었는데도 그렇네... 
<br>
`python replace space not working`으로 검색해보았더니 [이런](https://stackoverflow.com/questions/4369159/replace-n-with-br) 답변을 찾아볼 수 있었다. 

str을 해주라는 말인듯?

In [ ]:
df_copy['집행지명'] = df_copy['집행지명'].str.replace(' ', '')

In [ ]:
name_arr = df_copy['집행지명'].unique()
name_arr

제대로 띄어쓰기가 없어졌다!
다시 top20을 내보자

In [ ]:
df_copy.groupby('집행지명').sum().sort_values(by='cnt', ascending=False)[:5]

- '파리바게[트뜨]', '스타벅스'는 빼야겠다
- '시청'이 들어가는 곳도 빼야겠어
- year, month, day는 빼자

In [ ]:
df_copy = df_copy[df_copy['집행지명'].str.contains('파리바게[트뜨]') == False]
df_copy = df_copy[df_copy['집행지명'].str.contains('시청') == False]
df_copy = df_copy[df_copy['집행지명'].str.contains('스타벅스') == False]

In [ ]:
df_copy.groupby('집행지명')['집행지명', '집행금액', 'cnt'].sum().sort_values(by='cnt', ascending=False)[:5]

<br>
30위까지 찍어보니 '카페'/'커피' 키워드도 없어도 될 것 같다. 

In [ ]:
df_copy = df_copy[df_copy['집행지명'].str.contains('카페') == False]
df_copy = df_copy[df_copy['집행지명'].str.contains('커피') == False]

In [ ]:
df_copy.groupby('집행지명')['집행지명', '집행금액', 'cnt'].sum().sort_values(by='cnt', ascending=False)[:10]

In [ ]:
df_copy.groupby('집행지명')['집행지명', '집행금액', 'cnt'].sum().sort_values(by='집행금액', ascending=False)[:10]

In [ ]:
df_copy.to_csv('tidy_1.csv', index=False)

이상하다. 
서울시장 직속 부서만 크롤링 했을 때
참숯골이 cnt가 400가 넘었는데
전 부서 데이터로 분석했는데 
376회로 줄어들었다.

지난번 데이터로 해보자

In [ ]:
df_data_before = pd.read_csv('C:/Users/jeeyun/Downloads/crawl_data.csv')

In [ ]:
df_data_before.groupby('집행지명')['집행지명', '집행금액', 'cnt'].sum().sort_values(by='cnt', ascending=False)[:10]

In [ ]:
print('2017.8-2018.7전부서')
print(df_copy.shape)
print('2017년 직속부서')
print(df_data_before.shape)

기간이 달라졌지용! 기록기록

In [17]:
df_data_tidy_final = pd.read_csv('tidy_2.csv')

,1소속부서,구분,대상인원,집행금액,집행목적,집행지주소,집행지명,cnt
0,서울특별시장,시정 관련 간담회 등,시장 등 12명,344000.0,시정 정책추진 자문 간담회,서초구 강남대로 213,동신참치양재직,1
1,서울특별시장,시정 관련 간담회 등,시장 등 8명,226000.0,시정 주요 경제정책 현안추진 의견수렴 간담회,종로구 북촌로,한뫼촌,1
2,서울특별시장,시정 관련 간담회 등,시장 등 2명,60000.0,시정 남북교류협력관련 자문회의,종로구 자하문로 31-10,삼다도,1
3,서울특별시장,시정 관련 간담회 등,시장 등 6명,136000.0,시정 정책 설명 및 자문 간담회,강동구 강동대로,（주）신화푸드긴자,1
4,서울특별시장,시정 관련 간담회 등,시장 등 3명,75000.0,시정 정책 설명 및 의견수렴 언론 간담회,중구 통일로4길 22,어상,1
5,서울특별시장,시정 관련 간담회 등,시장 등 8명,160000.0,시정 정책현안 의견수렴 간담회,금천구 시흥대로 414,마쯔,1
6,서울특별시장,시정 관련 간담회 등,시장 등 14명,420000.0,시정 정책 정부협력 간담회,종로구 재동,（주）달개비자연음식,1
7,서울특별시장,시정 관련 간담회 등,시장 등 7명,181000.0,시정 정책 추진 협의 간담회,마포구 백범로 23,주식회사거구상사,1
8,서울특별시장,시정 관련 간담회 등,시장 등 8명,180000.0,시정정책 추진관련 종교계 의견 수렴 간담회,강북구 한천로 1153,자연채샤브,1
9,서울특별시장,시정 관련 간담회 등,시장 등 3명,70000.0,시정 정책추진 자문 간담회,종로구 자하문로10길,태진복집,1


In [34]:
df_cnt = df_data_tidy_final.groupby('집행지명')['1소속부서','구분','대상인원','집행금액','집행목적','집행지주소','집행지명','cnt'].sum().sort_values(by='cnt', ascending=False)[:100]
df_temp = df_data_tidy_final.groupby('집행지명')['집행지명', 'cnt'].sum().sort_values(by='cnt', ascending=False)[:100]
df_cnt['집행지'] = df_temp.index
df_cnt.to_excel('C:\workspace\djangoPrj\popular_restaurants_from_officials/cnt_rank.xlsx', index=False)

FileLinks('C:\workspace\djangoPrj\popular_restaurants_from_officials')

C:\workspace\djangoPrj\popular_restaurants_from_officials/
  cnt_rank.xlsx
  official_div.json
  raw.csv
  README.md
  tidy_1.csv
  tidy_2.csv
  tidy_data.csv
  tidy_data.ipynb
  web_crawling.ipynb
  공무원지도_피드백.md
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git/
  COMMIT_EDITMSG
  config
  description
  FETCH_HEAD
  HEAD
  index
  ORIG_HEAD
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\hooks/
  applypatch-msg.sample
  commit-msg.sample
  fsmonitor-watchman.sample
  post-update.sample
  pre-applypatch.sample
  pre-commit.sample
  pre-push.sample
  pre-rebase.sample
  pre-receive.sample
  prepare-commit-msg.sample
  update.sample
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\info/
  exclude
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\logs/
  HEAD
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\logs\refs\heads/
  branch2
  dev
  master
  top10_release
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\logs\refs\remotes\origin/
  dev
  master
  top10_release
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\objects\00/
  afd8e82484e9b28cface83d193c84805e46036
  f9038ed0ee44f964304c86a3d5d01987e71fca
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\objects\01/
  103285080a931f8dfaae733ce242d772bb9c18
  a688294847d7286d9359d325a03443552661b4
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\objects\02/
  b0bbde2ec388bf85317b5d8a87db71f8a75145
  f258f9289a5382860b61dc0970673704cd36c6
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\objects\03/
  129eebc3b498a24c8a2bc631969f4f5a88d57d
  357ce4f5833006114f2e790a7b89dbf69d4f90
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\objects\04/
  53ce87932b016154c6037d89c4675a697766bc
  73643c419af10e77427f5841c8d773d513e538
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\objects\05/
  f3a6e5eb189936b17e93f28e86dc781e800f04
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\objects\06/
  07bf2e5e346dec24ef9d51d57ad7fc73c16576
  cfe42c68a7e5255cd1171e9ec4e1897a8678b9
  e334a0e9ef6379fe29aac23a2fd3d0ef012043
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\objects\07/
  a0da9c794d542807c9f6639968b5bcfb1b3a9a
  b57cbd4d579fbfca61ed625dcb90b1292f20e3
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\objects\09/
  4d8da70f377b8812f07aa6a14137d27400cf55
  7a86c699a42ed9e5b071b3b98cdee0abb9eafc
  c95b0f6aa9489deed790bd022d245cdb5037dc
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\objects\0a/
  096502da9b95076187df64b7f06376429c4a17
  a4b144b7d9d0cc947e4efe298b1fade7eb3c14
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\objects\0b/
  36db952d73da2bcce9711406e504e6faa86a3a
  3c1233a6d95312bea98b79a94141a1794853ad
  9d1520cf413757ee262c3aef902e684dcd77bf
  cad6b506bd1e18b0144883d7f383e469077ef0
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\objects\0c/
  1de59b9bcc03dc06004cd915d7c1c6b4ff0b81
  28204f50c2516dddd9650c09b03de7987db2e9
  9f1fc9bbe35bf40707769cd6095028358a088c
  bda561b9598626ea42cd5453b3ef69e69f3c0c
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\objects\0d/
  070d2af1cb403b96adec78f49f9e14d05914c6
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\objects\0f/
  435abcc307c02f8583856c9d37400a6d9965f5
  63721d56112403a7ae7968874a14d2ff83ca8a
  b52b22db260570b6937cfb123c1ddab06f0350
  ca6f22bc07a1610bc824febf48f70ef6867079
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\objects\10/
  a9e05be7c8f4b9b63fa1b8a4492cf9dd6e67ea
  f525538bfd568f87e3f54e156bed18f94c599a
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\objects\11/
  5764265be832cfb4cd2fddc5acb21fb98b4785
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\objects\13/
  48eec19bd4547ea0fbd818166ef20895e72ddb
  c56b2a73f323af6eab968f93e05500b0c1a875
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\

In [37]:
df_money = df_data_tidy_final.groupby('집행지명')['1소속부서','구분','대상인원','집행금액','집행목적','집행지주소','집행지명','cnt'].sum().sort_values(by='집행금액', ascending=False)[:100]
df_temp = df_data_tidy_final.groupby('집행지명')['집행지명', '집행금액'].sum().sort_values(by='집행금액', ascending=False)[:100]
df_money['집행지'] = df_temp.index
df_money
# df_money.to_csv('C:\workspace\djangoPrj\popular_restaurants_from_officials/pay_rank.csv', index=False)
df_money.to_excel('C:\workspace\djangoPrj\popular_restaurants_from_officials/pay_rank.xlsx', index=False)

FileLinks('C:\workspace\djangoPrj\popular_restaurants_from_officials')

C:\workspace\djangoPrj\popular_restaurants_from_officials/
  cnt_rank.xlsx
  official_div.json
  pay_rank.xlsx
  raw.csv
  README.md
  tidy_1.csv
  tidy_2.csv
  tidy_data.csv
  tidy_data.ipynb
  web_crawling.ipynb
  공무원지도_피드백.md
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git/
  COMMIT_EDITMSG
  config
  description
  FETCH_HEAD
  HEAD
  index
  ORIG_HEAD
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\hooks/
  applypatch-msg.sample
  commit-msg.sample
  fsmonitor-watchman.sample
  post-update.sample
  pre-applypatch.sample
  pre-commit.sample
  pre-push.sample
  pre-rebase.sample
  pre-receive.sample
  prepare-commit-msg.sample
  update.sample
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\info/
  exclude
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\logs/
  HEAD
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\logs\refs\heads/
  branch2
  dev
  master
  top10_release
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\logs\refs\remotes\origin/
  dev
  master
  top10_release
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\objects\00/
  afd8e82484e9b28cface83d193c84805e46036
  f9038ed0ee44f964304c86a3d5d01987e71fca
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\objects\01/
  103285080a931f8dfaae733ce242d772bb9c18
  a688294847d7286d9359d325a03443552661b4
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\objects\02/
  b0bbde2ec388bf85317b5d8a87db71f8a75145
  f258f9289a5382860b61dc0970673704cd36c6
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\objects\03/
  129eebc3b498a24c8a2bc631969f4f5a88d57d
  357ce4f5833006114f2e790a7b89dbf69d4f90
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\objects\04/
  53ce87932b016154c6037d89c4675a697766bc
  73643c419af10e77427f5841c8d773d513e538
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\objects\05/
  f3a6e5eb189936b17e93f28e86dc781e800f04
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\objects\06/
  07bf2e5e346dec24ef9d51d57ad7fc73c16576
  cfe42c68a7e5255cd1171e9ec4e1897a8678b9
  e334a0e9ef6379fe29aac23a2fd3d0ef012043
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\objects\07/
  a0da9c794d542807c9f6639968b5bcfb1b3a9a
  b57cbd4d579fbfca61ed625dcb90b1292f20e3
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\objects\09/
  4d8da70f377b8812f07aa6a14137d27400cf55
  7a86c699a42ed9e5b071b3b98cdee0abb9eafc
  c95b0f6aa9489deed790bd022d245cdb5037dc
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\objects\0a/
  096502da9b95076187df64b7f06376429c4a17
  a4b144b7d9d0cc947e4efe298b1fade7eb3c14
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\objects\0b/
  36db952d73da2bcce9711406e504e6faa86a3a
  3c1233a6d95312bea98b79a94141a1794853ad
  9d1520cf413757ee262c3aef902e684dcd77bf
  cad6b506bd1e18b0144883d7f383e469077ef0
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\objects\0c/
  1de59b9bcc03dc06004cd915d7c1c6b4ff0b81
  28204f50c2516dddd9650c09b03de7987db2e9
  9f1fc9bbe35bf40707769cd6095028358a088c
  bda561b9598626ea42cd5453b3ef69e69f3c0c
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\objects\0d/
  070d2af1cb403b96adec78f49f9e14d05914c6
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\objects\0f/
  435abcc307c02f8583856c9d37400a6d9965f5
  63721d56112403a7ae7968874a14d2ff83ca8a
  b52b22db260570b6937cfb123c1ddab06f0350
  ca6f22bc07a1610bc824febf48f70ef6867079
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\objects\10/
  a9e05be7c8f4b9b63fa1b8a4492cf9dd6e67ea
  f525538bfd568f87e3f54e156bed18f94c599a
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\objects\11/
  5764265be832cfb4cd2fddc5acb21fb98b4785
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\objects\13/
  48eec19bd4547ea0fbd818166ef20895e72ddb
  c56b2a73f323af6eab968f93e05500b0c1a875
C:\workspace\djangoPrj\popular_restaurants_from

,집행금액,cnt
집행지명,,
우도일식,42050800.0,308
참숯골,34195380.0,264
구이구이,23646200.0,250
곰국시집,26824350.0,233
동해일식,29497000.0,169
창고43,18432300.0,140
오리마당,15320500.0,139
바닷가작은부엌,19377500.0,134
무교소호정,14603000.0,132
